# Fake product detect from shopee

## scrape machine

### import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import bs4
import pandas as pd

In [2]:
url_L=[]
name_L=[]
price_L=[]
sold_L=[]
amount_reviewer_L=[]
all_store_reviewer_L=[]
all_product_store_L=[]
response_rate_L=[]
follower_L=[]
joined_L=[]
rating_L=[]
new_url_L=[]

### Set up the driver

In [3]:
s = Service('F:\Code\edgedriver_win64 (3)\msedgedriver.exe') #your path driver
driver=webdriver.Edge(service=s)#choose the driver ex.Edge, Firefox
link=f"https://shopee.co.th/search?keyword=airpod&page=1 " #insert the shopee url
driver.get(link)

#change language to english in shopee
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,f"/html/body/div[2]/div[1]/div[1]/div/div[3]/div[2]/button")))
eng_button=driver.find_element(By.XPATH,"/html/body/div[2]/div[1]/div[1]/div/div[3]/div[2]/button")
eng_button.click()

### Functions

In [ ]:
wait=WebDriverWait(driver, 10)
def k_and_m_to_float(txt):
    "ฟังก์ชันเพื่อเปลี่ยน k เป็น 1000 เปลี่ยน m เป็น 1000000 เปลี่ยนเป็น float, this function change k to multiple 100 and m to multiple 1000000 "
    if txt[-1]=="k":
        txt=(float(txt[:-1])*1000)
    elif txt[-1]=="m":
        txt=(float(txt[:-1])*1000000)
    else:
        txt=(float(txt))
    return txt

def hours_convert(txt):
    "ฟังก์ชันเพื่อเปลี่ยนทุกหน่วยเป็นหน่วยชั่วโมง,change, this function change all time unit in to hours"
    if txt[-10:]==' years ago':
        txt=float(txt[:-10])*360*24
    elif txt[-10:]==' hours ago':
        txt=float(txt[:-10])
    elif txt[-11:]==' months ago':
        txt=float(txt[:-11])*30*24
    elif txt[-9:]==' days ago':
        txt=float(txt[:-9])*24
    else:
        txt=(float(0))
    return txt

def waiter(datatype,positiondata):
    "datatype ex. By.CSS_SELECTOR ,positiondata ex. f'.pmmxKx'"
    wait.until(EC.visibility_of_element_located((datatype,positiondata)))
    global soup
    soup=bs4.BeautifulSoup(driver.page_source)

### Url archive page

In [ ]:
for N_of_page in range(1,12): #total of page 
    link=f"https://shopee.co.th/search?keyword=airpod&page={N_of_page}" #ex. "https://shopee.co.th/search?keyword=phone&page={N_of_page}"
    driver.get(link)
    driver.execute_script('document.body.style.zoom="1%"')
    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,f".row")))
        soup=bs4.BeautifulSoup(driver.page_source)
        url=soup.find_all('div', {'class':'col-xs-2-4 shopee-search-item-result__item'})
    except:
        print("not found a product row")
    for i in url:
        url_L.append(i.find('a').get('href'))
        print(len(url_L))

### Scrape Feature(X) 

In [ ]:
for i in range ((len(url_L))):
    w=('https://shopee.co.th'+url_L[i])
    new_url_L.append(w)
    print(len(new_url_L))
    driver.get(w)
    driver.execute_script('document.body.style.zoom="1%"')    
    soup=bs4.BeautifulSoup(driver.page_source)
    #ชื่อสินค้า Name
    try:
        waiter(By.CSS_SELECTOR,".VCNVHn")
        name_L.append(soup.find('div',{'class':'VCNVHn'}).text)
    except:
        name_L.append("None")
    #ราคา Price
    try:
        waiter(By.CSS_SELECTOR,f".pmmxKx")
        try:
            price_L.append(float(soup.find('div',{'class':'pmmxKx'}).text.replace(",",'').replace("฿","")))
        except:
            price_L.append(float((soup.find('div',{'class':'pmmxKx'}).text.replace(",",'').replace("฿","")).partition(' - ')[0]))
    except:
        price_L.append("None")
    #จำนวนที่ขาย sold
    try:
        waiter(By.CSS_SELECTOR,f"._45NQT5")
        sold=(soup.find('div',{'class':'_45NQT5'}).text)
        sold_L.append(k_and_m_to_float(sold))
    except:
        sold_L.append('None')
    #จำนวนคนรีวิว amount reviewer
    try:
        waiter(By.CSS_SELECTOR,f"div.HXGLyo:nth-child(2) > div:nth-child(1)")
        amount_reviewer=(((soup.find_all('div',{'class':'MrYJVA'}))[1]).text)
        amount_reviewer_L.append(k_and_m_to_float(amount_reviewer))
    except:
        amount_reviewer_L.append(float(0))
    #จำนวนคนรีวิวร้านค้าทั้งหมด all store reviewer
    try:
        waiter(By.CSS_SELECTOR,f"div.MSPJbO:nth-child(1) > div:nth-child(1) > span:nth-child(2)")
        all_store_reviewer=((soup.find_all('span',{'class':'_32ZDbL'}))[0].text)
        all_store_reviewer_L.append(k_and_m_to_float(all_store_reviewer))
    except:
        all_store_reviewer_L.append('None')
    #จำนวนสินค้าทั้งหมดของร้านค้า all product of store
    try:
        waiter(By.CSS_SELECTOR,f".g54jiy")
        all_product_store_L.append(k_and_m_to_float((soup.find('span',{'class':'_32ZDbL g54jiy'}).text)))
    except:
        all_product_store_L.append('None')
    #อัตราการตอบกลับของร้านค้า response rate of store
    try:
        waiter(By.CSS_SELECTOR,f"div.MSPJbO:nth-child(2) > div:nth-child(1) > span:nth-child(2)")
        response_rate_L.append(float((soup.find_all('span',{'class':'_32ZDbL'}))[2].text.replace('%','')))
    except:
        response_rate_L.append('None')
    #จำนวนผู้ติดตาม follower of store
    try:
        waiter(By.CSS_SELECTOR,f"div.MSPJbO:nth-child(3) > div:nth-child(2) > span:nth-child(2)")
        follower_L.append(k_and_m_to_float((soup.find_all('span',{'class':'_32ZDbL'}))[5].text))
    except:
        follower_L.append('None')
    #เข้าร่วมเมื่อกี่ชั่วโมงแล้ว joined
    try:
        waiter(By.CSS_SELECTOR,f"div.pHNb7U:nth-child(1) > span:nth-child(2)")
        joined_L.append(hours_convert((soup.find_all('span',{'class':'_32ZDbL'}))[4].text))
    except:  
        joined_L.append('None')
    #rating
    try:
        waiter(By.CSS_SELECTOR,f".Ga-lTj")
        rating_L.append(float((soup.find('div',{'class':'MrYJVA Ga-lTj'})).text))
    except:
        rating_L.append(float(0))

### Make to dataframe for use

In [ ]:
shopee=pd.DataFrame([new_url_L,name_L,price_L,sold_L,amount_reviewer_L,all_store_reviewer_L,all_product_store_L,response_rate_L,follower_L,joined_L,rating_L]).transpose()
shopee.columns=['url','name','price','sold','amount reviewer','all reviewer of store','amount product','response rate','follower','joined','rating']

In [ ]:
shopee.to_excel(r'F:\Code\Shopee_Data_Scraping\rawdata\earpod2.xlsx')

## Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
url = (r"https://github.com/JackJessada/fake_product_detect/raw/main/data/thirddataset.xlsx")
df = pd.read_excel(url)

In [ ]:
df.tail()

In [ ]:
X = df.drop(['ID', 'url', 'name', 'comment', 'Label'], axis=1)
y = df['Label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #split data

In [ ]:
ax = y_train.value_counts().plot.pie(autopct='%.2f')
_ = ax.set_title("data")

### over samplingsmote data

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote=SMOTE(sampling_strategy={0: 1424, 1: 3916, 2:1780},random_state=0,k_neighbors=7,)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
ax = y_train_sm.value_counts().plot.pie(autopct='%.2f')
_ = ax.set_title("Over-sampling(smote)")

## model

### choose models

โดยการ cross validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

In [ ]:
rf_clf = RandomForestClassifier()
xgb_clf=xgb.XGBClassifier()
knn_clf=KNeighborsClassifier()
skf = StratifiedKFold(10,shuffle=True)
for model in [rf_clf, xgb_clf, knn_clf]:
    print('Avg. R2- ' + str(model) + ': ' + str(np.mean(cross_val_score(model, X, y, cv=skf))))

### model

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
def classification_model(clf, X_train, y_train, X_test, y_test, show_feature_importance=True):
    ls_y_hat_train = []
    ls_y_train = []

    clf_model = clf.fit(X_train, y_train)
    y_hat_train = clf_model.predict(X_train)
    ls_y_hat_train.extend(y_hat_train)
    ls_y_train.extend(y_train)
    report_train = classification_report(ls_y_train, ls_y_hat_train)
    print('training set')
    print(report_train) 
    print('\n')
    ls_y_hat_test = []
    ls_y_test = []
    y_hat_test = clf_model.predict(X_test)
    ls_y_hat_test.extend(y_hat_test)
    ls_y_test.extend(y_test)
    report_test = classification_report(ls_y_test, ls_y_hat_test)
    print('test set')
    print(report_test) 
    print('\n')
    predicted = clf_model.predict(X_test)
    print(confusion_matrix(y_test, predicted))

    if show_feature_importance:
        try:
            print('Standardized Coeff.')
            print(pd.DataFrame({"Feature":X_train.columns.tolist(),"Coefficients":clf_model.coef_[0]}).to_string())
        except:
            try:
                print(pd.DataFrame({"Feature":X_train.columns.tolist(),"Coefficients":clf_model.feature_importances_}).to_string())
            except:
                print("No Standardized Coeff.")

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=50,max_depth=10, random_state=0)

In [ ]:
classification_model(rf_clf,X_train_sm,y_train_sm,X_test,y_test)

### dump model

In [ ]:
from joblib import dump
clf_model = rf_clf.fit(X_train_sm, y_train_sm)
dump(clf_model,'rf_clf_model.joblib')

## Explainer

In [ ]:
import shap
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
import seaborn as sns

In [ ]:
shap_values = shap.TreeExplainer(rf_clf).shap_values(X_train)

In [ ]:
shap.summary_plot(shap_values, X_train)

In [ ]:
predicted = rf_clf.predict(X_test)
j=(confusion_matrix(y_test, predicted))
sns.heatmap(j, annot=True, fmt='g')

In [ ]:
explainer = ClassifierExplainer(rf_clf, X_test, y_test, 
                               cats=X.columns,
                               labels=['0', '1','2'])

ExplainerDashboard(explainer).run()